<a href="https://colab.research.google.com/github/jantuitman/deeplearning/blob/main/extractor_model_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maak vragen van teksten.

Dit notebook gebruikt GPT-3 om vragen van teksten te maken.


In [1]:
!pip install openai
!pip install pandas
!pip install pyyaml


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.4-py3-none-any.whl size=67744 sha256=2b396e6269f2802e479fbbc0c3327ab3397d3a56d303f3e57c30698a9715c104
  Stored in directory: /root/.cache/pip/wheels/2b/d8/4e/268f029bd3277c1dd9e8781a0e0296e0a63822665bfa2429fc
Successfully built openai
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
# Wat algemene imports

import json
import pandas as pd
import yaml
import openai


# Squad v2 dataset

Dit is een functie die de squad v2 dataset (in een afwijkend formaat met nederlandse vertalingen er in) inleest in een array.Verderop gaan we die teksten verder opsplitsen in zinnen en paragrafen maar dat koppelen we niet hard aan Squad, zodat we het ook op de werkteksten kunnen toepassen.


In [3]:
# read a json file containing an array of objects
# each object has a "question" and "answer" field
# the question is the input and the answer is the output
# we will return a list of tuples
def read_squadv2_dataset(filename):
    with open(filename) as f:
        data = json.load(f)
        # data has the following structure:
        # data = {
        #     "paragraphs": [
        #         {
        #             "translated_context": "text",
        #             "translated_question": "text",
        #             "translated_answers": [
        #                 "text",
        #                 "text",
        #                 "text"
        #             ],
        #             "is_impossible": true
        #         },
        #         ...
        #     ]
        # }
        # if is_impossible is true, then translated_answers is empty and we want to return the answer "Onbekend"
        # if is_impossible is false, then translated_answers is not empty and we want to return the first answer
        # we will return a list of tuples
        # we need to make a question_text from the translated_context and translated_question
        result= []
        previous_found_context = None
        for paragraph in data['paragraphs']:
          if paragraph['translated_context'] != previous_found_context:
            if previous_found_context is not None:
              result.append(previous_found_context)
            previous_found_context = paragraph['translated_context']    
        if previous_found_context is not None:
          result.append(previous_found_context)
        return result

De stap hier beneden leest de eerste 10% van de squad v2 dataset van Jan uit Jans Google drive. TODO: verhuis deze dataset naar een locatie in dit-werkt.nl waar Google colab makkelijk bij kan.

Als handmatige stap benodigd totdat dat gedaan is: zorg dat je een google drive hebt en plaats een file die dev-nl.json heet in een map InputFiles in google drive.

In de linkerbalk op google colabs kun je je drive koppelen.


In [4]:
import pandas as pd


squad_texts = read_squadv2_dataset('drive/MyDrive/InputData/dev-nl.json')

# controle resultaten door het weer te geven als een pandas dataframe

pd.DataFrame(squad_texts,columns=["squad_paragraph"])

,squad_paragraph
0,De Normandiërs (Norman: Nourmands; Frans: Norm...
1,De Normandische dynastie had een grote politie...
2,"De Engelse naam ""Normans"" komt van de Franse w..."
3,In de loop van de 10e eeuw veranderden de aanv...
4,Voor de komst van Rollo verschilde de bevolkin...
...,...
2912,"waarin de massa van het voorwerp is, de snelhe..."
2913,Een conservatieve kracht die inwerkt op een ge...
2914,Voor bepaalde fysische scenario's is het onmog...
2915,Het verband tussen macroscopische niet-conserv...


# Functie om arrays van tekst te converteren naar tuples van zinnen en paragrafen.

Een tekst kan meerdere paragrafen bevatten gescheiden door newlines (\n tekens). Een paragraaf kan meerdere zinnen bevatten, gescheiden door punten, uitroeptekens en vraagtekens.

Het resultaat van de functie moet een lijst van tuples zijn.

Deze functie is een "poor mans parser" want hij kan niet overweg met ingewikkelde dingen zoals quotes in quotes, meerdere paragrafen binnen een quote, mensen die vergeten quotes af te sluiten, enz. enz.
In de toekomst kan de parser misschien vervangen worden door een AI model die parst. Maar voor nu is het niet zo belangrijk omdat we zelf wel controle hebben over de data die we er mee parsen.

```
[
  ('sentence','De eerste zin van de tekst.'),
  ('sentence','De tweede zin van de tekst.'),
  ('paragraph','De eerste paragraaf, die komt na alle zinnen van de eerste paragraaf.'),
  ('sentence','De derde zin van de tekst, die in de tweede paragraaf zit.'),
  ('sentence','De vierde zin van de tekst, die in de tweede paragraaf zit.'),
  ('paragraph','De tweede paragraaf, die komt na alle zinnen van de tweede paragraaf.'),
  ...
]

```

In [5]:
import re

def split_text(text):
    result = []
    paragraphs = text.split('\n')
    for i, paragraph in enumerate(paragraphs):
        sentence = ""
        in_quote = False
        for c in paragraph:
            if c == '"' or c == "'":
                in_quote = not in_quote
            if c in ".!?":
                if not in_quote:
                    if sentence != "":
                        sentence += c
                        result.append(('sentence', sentence.strip(),paragraph))
                        sentence = ""
            else:
                sentence += c
        if sentence != "":
            result.append(('sentence', sentence.strip(),paragraph))
        result.append(('paragraph',paragraph, None))    
    return result

In [6]:
# de test of het werkt.
split_text("'Geloof je dat echt? Ik ben het er niet mee eens', zei hij. Daarna ging hij naar de winkel.\nDit is een tekst met meerdere punten... En nog een zinnetje!\nGaat alles goed?")

[('sentence',
  "'Geloof je dat echt Ik ben het er niet mee eens', zei hij.",
  "'Geloof je dat echt? Ik ben het er niet mee eens', zei hij. Daarna ging hij naar de winkel."),
 ('sentence',
  'Daarna ging hij naar de winkel.',
  "'Geloof je dat echt? Ik ben het er niet mee eens', zei hij. Daarna ging hij naar de winkel."),
 ('paragraph',
  "'Geloof je dat echt? Ik ben het er niet mee eens', zei hij. Daarna ging hij naar de winkel.",
  None),
 ('sentence',
  'Dit is een tekst met meerdere punten.',
  'Dit is een tekst met meerdere punten... En nog een zinnetje!'),
 ('sentence',
  'En nog een zinnetje!',
  'Dit is een tekst met meerdere punten... En nog een zinnetje!'),
 ('paragraph',
  'Dit is een tekst met meerdere punten... En nog een zinnetje!',
  None),
 ('sentence', 'Gaat alles goed?', 'Gaat alles goed?'),
 ('paragraph', 'Gaat alles goed?', None)]

# GPT 3 prompts voor het verwerken van de sentences en paragrafen

Deze prompt gebruiken we om aan GPT3 te vertellen hoe we een zin willen omvormen naar vragen over de zin. We hebben een aparte prompt voor paragrafen.

In [30]:
import itertools
data_set = [split_text(text) for text in squad_texts]
data_set = list(itertools.chain.from_iterable(data_set))
None

#GPT functie om 1 tuple te verwerken.

In [24]:
def genereer_vragen(text):
  prompt = f"""
  
      INSTRUCTIE: Verzin 5 vragen waarvan het antwoord in onderstaande tekst gegeven is.
      TEKST: Een van de eerste Normandische huurlingen die als Byzantijnse generaal diende was Hervé in de jaren 1050. Tegen die tijd dienden er echter al Normandische huurlingen tot in Trebizond en Georgië. Zij waren gelegerd in Malatya en Edessa, onder de Byzantijnse hertog van Antiochië, Isaac Komnenos.  

      VRAGEN:
      1. Wie was een van de eerste Normandische huurlingen die als Byzantijnse generaal diende?
      2. In welke jaren diende Hervé als Byzantijnze generaal.
      3. In welke streken of landen dienden ook al Normandische huurlingen toen Hervé als Byzantijnse generaal diende?
      4. In welke steden waren Normandische huurlingen gelegerd die dienden onder Isaac Komnenos?
      5. Hoe heette de Byzantijnze hertog van Antiochië?


      INSTRUCTIE: Verzin 5 vragen waarvan het antwoord in onderstaande tekst gegeven is.
      TEKST: Een medewerker tuincentrum verkoopt artikelen in een tuincentrum, zoals bloemen, planten en tuinartikelen. Voorbeelden van werkzaamheden van de medewerker tuincentrum: helpt klanten, adviseert over producten, verzorgt bloemen en planten, maakt eenvoudige boeketten en bindt ze, laat goederen aan de kassa afrekenen, etaleert, houdt verkoopruimtes schoon en behandelt klachten.
 
      VRAGEN:
      1. Wat is een medewerker tuincentrum?
      2. Wat verkoopt een medewerker tuincentrum?
      3. Wat zijn voorbeelden van werkzaamheden van een medewerker tuincentrum?
      4. Wat verzorgt een medewerker tuincentrum?
      5. Moet een mederwerker tuincentrum ook klachten behandelen? 

      INSTRUCTIE: Verzin 5 vragen waarvan het antwoord in onderstaande tekst gegeven is.
      TEKST:{text}
  """


  completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=["TEKST","INSTRUCTIE"],
        temperature=0.7,
        api_key=api_key
  )
  generated_text = completions.choices[0].text
  lines = generated_text.split("\n")

  is_question = lambda text: bool(re.match(r'^\s*\d+[.][ ].*\?\s*$', text))
  extract_question = lambda text: re.sub(r'^\s*\d+[.]\s*', '', text).strip()
  questions = list(filter(is_question, lines))
  questions_texts = [extract_question(question) for question in questions]

  return questions_texts

In [9]:
def read_secrets():
    with open("drive/MyDrive/InputData/secrets.yml", 'r') as file:
        data = yaml.safe_load(file)
        return data


In [13]:
secrets = read_secrets()
api_key = secrets['open-api']['key']
model = 'text-davinci-003'

In [28]:
results = []
for item in data_set[:20]:
  text_type, text,x = item
  if text_type == "paragraph":
    questions=genereer_vragen(text)
    results.append({ 'context': text, 'questions': questions})

print(json.dumps(results,indent=4))

[
    {
        "context": "De Normandi\u00ebrs (Norman: Nourmands; Frans: Normands; Latijn: Normanni) waren de mensen die in de 10e en 11e eeuw hun naam gaven aan Normandi\u00eb, een regio in Frankrijk. Zij stamden af van Noorse (\"Norman\" komt van \"Noorman\") rovers en piraten uit Denemarken, IJsland en Noorwegen die onder hun leider Rollo trouw zwoeren aan koning Karel III van West-Frankrijk. Door generaties van assimilatie en vermenging met de inheemse Frankische en Romeins-Gaulische bevolking, zouden hun afstammelingen geleidelijk samensmelten met de op Karolingische leest geschoeide culturen van West-Frankrijk. De aparte culturele en etnische identiteit van de Noormannen ontstond aanvankelijk in de eerste helft van de 10e eeuw en bleef zich in de daaropvolgende eeuwen ontwikkelen.",
        "questions": [
            "Vanwaar komt de naam Normandi\u00eb?",
            "Uit welke landen stamden de Normandi\u00ebrs af?",
            "Wie was de leider van de Normandi\u00ebrs?",
 

In [33]:
import json

with open('drive/MyDrive/OutputModels/questions2.json', 'r') as file:
    array = json.load(file)


results = array
for count,item in enumerate(data_set):
  text_type, text,x = item
  if count < 4600:
    continue
  if text_type == "paragraph":
    questions=genereer_vragen(text)
    results.append({ 'context': text, 'questions': questions})
  if count == 5 or (count % 50 == 0):
    print("saving",count)
    with open('drive/MyDrive/OutputModels/questions.json', 'w') as file:
      json.dump(results,file,indent=4)

saving 4600
saving 4650
saving 4700
saving 4750
saving 4800
saving 4850
saving 4900
saving 4950
saving 5000
saving 5050
saving 5100
saving 5150
saving 5200
saving 5250
saving 5300
saving 5350
saving 5400
saving 5450
saving 5500
saving 5550
saving 5600
saving 5650
saving 5700
saving 5750
saving 5800
saving 5850
saving 5900
saving 5950
saving 6000
saving 6050
saving 6100
saving 6150
saving 6200
saving 6250
saving 6300
saving 6350
saving 6400
saving 6450
saving 6500
saving 6550
saving 6600
saving 6650
saving 6700
saving 6750
saving 6800
saving 6850
saving 6900
saving 6950
saving 7000
saving 7050
saving 7100
saving 7150
saving 7200
saving 7250
saving 7300
saving 7350
saving 7400
saving 7450
saving 7500
saving 7550
saving 7600
saving 7650
saving 7700
saving 7750
saving 7800
saving 7850
saving 7900
saving 7950
saving 8000
saving 8050
saving 8100
saving 8150
saving 8200
saving 8250
saving 8300
saving 8350
saving 8400
saving 8450
saving 8500
saving 8550
saving 8600
saving 8650
saving 8700
savi

ServiceUnavailableError: ignored